In [25]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, pooling
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import EarlyStopping
import pandas as pd, numpy as np
from sklearn.preprocessing import RobustScaler
import random
from numpy import *

np.random.seed(30)

test1=pd.read_csv('scared.csv', header=None)
test2=pd.read_csv('newnor.csv', header=None)

#test1=pd.read_csv('normal4.csv')
#test2=pd.read_csv('emer4.csv')

#test1 = test1.drop(["Time", "Unnamed: 0"], axis=1)
#test2 = test2.drop(["Time", "Unnamed: 0"], axis=1)
test1.drop(test1.columns[[0]], axis='columns')
test2.drop(test1.columns[[0]], axis='columns')


#test1.head()
#test2.head()

,1,2,3,4,5,6,7,8,9
0,51417,4070,7453,8840,5989,1521,524,34,41
1,83916,17563,6381,17881,6470,2237,1531,29,38
2,36465,11288,25116,2488,2558,3378,1633,51,29
3,42232,1512,6986,4969,4562,5628,1636,47,30
4,15630,6390,3575,7721,1515,1648,943,61,26
...,...,...,...,...,...,...,...,...,...
4875,478517,36260,175578,61774,43611,36125,20340,56,56
4876,383650,37270,85342,46733,32943,68735,21614,63,51
4877,217830,43343,7003,7070,4668,14239,6100,51,35
4878,266131,15031,34221,17515,29910,24582,7709,41,27


In [26]:
test1["label"] = "unsafe"
test2["label"] = "safe"

X=pd.concat([test1, test2])


#print(X.shape)
#print(X)
#"""
#Y를 범주형 데이터로 전환
bclass = {'safe':[1, 0], 'unsafe':[0, 1]}
y = np.empty((len(X), 2))
for i, v in enumerate(X['label']):
    y[i] = bclass[v]

#X축의 label을 제외한 테스트 데이터 셋
del X['label']
#X = X[['delta','theta','lowAlpha','highAlpha','lowBeta','highBeta','lowGamma','midGamma']].to_numpy()
X=X.to_numpy()
scaler = RobustScaler()
scaler.fit(X)
X = scaler.transform(X)
X.shape
#"""

(5679, 10)

In [27]:
tmp = [[x,y_tmp] for x, y_tmp in zip(X, y)]
random.shuffle(tmp)
X= [n[0] for n in tmp]
y = [n[1] for n in tmp]

X = np.asarray(X, dtype=np.float32)
y = np.asarray(y, dtype=np.float32)

#print(X.shape, y.shape)

In [28]:
#scaler = RobustScaler()
#scaler.fit(X)
#X = scaler.transform(X)
#print(X)
#np.mean(X), np.std(X)

In [29]:
x_train, y_train = X[1:600], y[1:600]
x_test, y_test = X[600:], y[600:]

In [47]:
x_train = x_train.reshape(x_train.shape[0], 10, 1)
x_test = x_test.reshape(x_test.shape[0], 10, 1)
print(x_train)
print('x_train shape:', x_train.shape[1])
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(y_train.shape[1])

[[[-0.46304196]
  [-0.41419405]
  [-0.3056084 ]
  ...
  [-0.3495389 ]
  [ 0.5       ]
  [ 0.4       ]]

 [[ 0.83859956]
  [ 0.20619962]
  [-0.35156408]
  ...
  [-0.13000269]
  [-1.3181819 ]
  [-0.65      ]]

 [[-0.2905138 ]
  [ 2.3843274 ]
  [ 1.59546   ]
  ...
  [ 0.42617962]
  [-0.13636364]
  [-0.1       ]]

 ...

 [[ 0.11363488]
  [ 0.3941317 ]
  [ 0.13393947]
  ...
  [ 0.11710986]
  [ 0.04545455]
  [-0.85      ]]

 [[-0.21238564]
  [ 0.7993516 ]
  [ 0.03278131]
  ...
  [-0.24764974]
  [ 1.5454545 ]
  [-0.15      ]]

 [[-0.4216414 ]
  [-0.13649923]
  [ 0.09209692]
  ...
  [-0.29098397]
  [ 0.5       ]
  [-0.15      ]]]
x_train shape: 10
599 train samples
5079 test samples
2


In [52]:
def evaluate_model(x_train, y_train, x_test, y_test):
    model = Sequential()
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    model.add(Convolution1D(filters=64, kernel_size=3, activation='relu',
                        input_shape=(n_timesteps,n_features)))
    model.add(Convolution1D(filters=64, kernel_size = 3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_outputs, activation='softmax'))

    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    hist=model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

    _, accuracy= model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
    return model

In [53]:
repeats=10
scores = list()
for r in range(repeats):
	model = evaluate_model(x_train, y_train, x_test, y_test)
# 	score = score * 100.0
# 	print('>#%d: %.3f' % (r+1, score))
# 	scores.append(score)
    
# summarize results
# print(scores)
# m, s = mean(scores), std(scores)
# print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

5079/5079 [==============================] - 0s 22us/step


In [55]:
dat=pd.read_csv('emer_test.csv')

# dat.drop(['Unnamed: 0'], axis=1, inplace=True)
# print(dat)
X = dat.to_numpy()
X = scaler.transform(X)
X = X.reshape(X.shape[0], 10, 1)
for i in range(0,len(X)):
    yhat = model.predict_classes(X[i:i+1])
    if yhat == [0]:
        print("safe")
    else :
        print("unsafe")


safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
safe
